In [1]:
import os
import json
from tqdm import tqdm
import numpy as np
import plotly.express as px

In [2]:
map_price_dir_path = './app/files/json/map/hn/street'
files = os.listdir(map_price_dir_path)

# data_list = [json.load(open(f'{map_price_dir_path}/{i}.json', encoding='utf-8')) for i in range(1983)]
data_list = json.load(open('./app/files/json/map/hn/refined_price_map_data_by_street.json', encoding='utf-8'))

In [20]:
data_list[0]

{'time': '2023-10-23 23:08:07.453407',
 'input': {'CITY': 'Hà Nội',
  'DISTRICT': 'Ba Đình',
  'STREET': 'Kim Mã Thượng',
  'URI_REQ': 'kim-ma-thuong-cong-vi-ba-dinh-ha-noi',
  'LNG': '105.8136389',
  'LAT': '21.0358105',
  'WARD': 'Cống Vị',
  'NAME_OSM': 'Ngõ 35 Phố Kim Mã Thượng, Phường Cống Vị, Quận Ba Đình, Hà Nội, 11150, Việt Nam',
  'ID_OSM': 155790507},
 'output': {'alleyHousePrice': {'1': {'metadata': 'Ngõ loại 1',
    'min': 81.25569721545178,
    'max': 487.4785426121431,
    'mean': 235.82974430863163},
   '2': {'metadata': 'Ngõ loại 2',
    'min': 56.87898805081624,
    'max': 341.2349798285001,
    'mean': 165.08082101604214},
   '3': {'metadata': 'Ngõ loại 3',
    'min': 45.50319044065299,
    'max': 272.9879838628001,
    'mean': 132.0646568128337}},
  'streetHousePrice': {'metadata': 'Nhà ở mặt đường',
   'min': 123.27775986533993,
   'max': 609.3481782651788,
   'mean': 294.78718038578955}}}

In [34]:
lat_list = [data["input"]["LAT"] for data in data_list]
lon_list = [data["input"]["LNG"] for data in data_list]
street_price_list = [data["output"]["streetHousePrice"]["mean"] for data in data_list]
text_list = [f'Alley Price: {str([data["output"]["alleyHousePrice"]["1"]["mean"], data["output"]["alleyHousePrice"]["2"]["mean"], data["output"]["alleyHousePrice"]["3"]["mean"]])}' for data in data_list]
location_df = pd.DataFrame()
location_df['lat'] = lat_list
location_df['lon'] = lon_list
location_df['alley_price'] = text_list
location_df['street_price'] = street_price_list
location_df = location_df.reset_index().rename(columns = {'index': 'street_idx'})

<IPython.core.display.Javascript object>

In [35]:
location_df = location_df.sort_values(by = ['lat', 'lon'])
location_df

,street_idx,lat,lon,alley_price,street_price
941,941,20.9047245,105.8529086,"Alley Price: [75.96222413499785, 53.1735568944...",94.952780
942,942,20.9106863,105.8667762,"Alley Price: [64.76175096083797, 45.3332256725...",80.952189
930,930,20.9119974,105.8314832,"Alley Price: [66.60264479991785, 46.6218513599...",83.253306
936,936,20.9120906,105.8699312,"Alley Price: [64.93244224050062, 45.4527095683...",81.165553
1003,1003,20.9123119,105.8916039,"Alley Price: [65.56933845824352, 45.8985369207...",81.961673
...,...,...,...,...,...
502,502,21.090957,105.787416,"Alley Price: [117.38494000803279, 82.169458005...",146.731175
1109,1109,21.0918031,105.7775876,"Alley Price: [109.67158379706939, 76.770108657...",137.089480
558,558,21.091928,105.783417,"Alley Price: [95.305168986389, 66.713618290472...",119.131461
1112,1112,21.093938,105.777619,"Alley Price: [117.69007159669599, 82.383050117...",147.112589


In [36]:
location_df['lon'] = location_df['lon'].astype(np.float32)
location_df['lat'] = location_df['lat'].astype(np.float32)

In [37]:

color_scale = [(0, 'orange'), (1,'red')]

fig = px.scatter_mapbox(location_df, lat='lat', lon="lon", text = location_df['alley_price'].to_numpy(), color="street_price", color_continuous_scale=[[0, 'green'], [0.5, 'red'], [1.0, 'rgb(0, 0, 255)']],
                                zoom=8, size_max=10, opacity=1)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [38]:
meeyland_df = pd.read_csv('private_house_meey_land_location.csv')
meeyland_df = meeyland_df.groupby(['lat', 'lon'])['street_price'].mean().reset_index()
meeyland_df

<IPython.core.display.Javascript object>

,lat,lon,street_price
0,20.639435,105.763281,31.4800
1,20.665774,105.763376,7.0000
2,20.669425,105.704660,3.4000
3,20.678540,105.751608,19.6300
4,20.681223,105.758913,26.9100
...,...,...,...
3309,21.270090,105.855467,7.5600
3310,21.272368,105.749768,21.5032
3311,21.276614,105.819041,11.4150
3312,21.283399,105.723166,111.5000


In [39]:


color_scale = [(0, 'orange'), (1,'red')]

fig = px.scatter_mapbox(meeyland_df, lat='lat', lon="lon", color="street_price", color_continuous_scale=[[0, 'green'], [0.5, 'red'], [1.0, 'rgb(0, 0, 255)']],
                                zoom=8, size_max=10, opacity=1)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()